In [1]:
import collections
import os
import pickle
import random
import numpy as np
import math
import torch
import random

In [2]:
# def get_sets(path):
#     """
#     Parses the raw FB text files and returns the set of all entities(their machine ids) and relations
    
#     !!!The entities are sorted before assigning them to an index
    
#     Input: path object
    
#     Output: dictionaries of unique integer index to each entity and relation, set of edges for each split
#     """
#     entities, relations = set(), set()
#     edge_set = {}
#     for split in ["train.txt", "valid.txt", "test.txt"]:
#         with open(os.path.join(path, split), "r") as lines:
#             edges = set()
#             for line in lines:
#                 lhs, rel, rhs = line.strip().split("\t")
#                 entities.add(lhs)
#                 entities.add(rhs)
#                 relations.add(rel)
#                 edges.add((lhs,rel,rhs))
#         edge_set[split] = edges
    
#     ent_id = {k:i for (i,k) in enumerate(sorted(entities))}
#     rel_id = {k:i for (i,k) in enumerate(sorted(relations))}
                
#     return ent_id, rel_id, edge_set


In [3]:
def get_sets(path, edge_filter = False, max_edges = None):
    """
    Parses the raw FB text files and returns the set of all entities(their machine ids) and relations
    
    !!!The entities are sorted before assigning them to an index
    
    Input: path object
    
    Output: dictionaries of unique integer index to each entity and relation, set of edges for each split
    """
    entities, relations = set(), set()
    edge_set = {}
    for split in ["train.txt", "valid.txt", "test.txt"]:
        with open(os.path.join(path, split), "r") as lines:
            edges = set()
            for line in lines:
                lhs, rel, rhs = line.strip().split("\t")
                entities.add(lhs)
                entities.add(rhs)
                relations.add(rel)
                edges.add((lhs,rel,rhs))
        edge_set[split] = edges
    
    ent_id = {k:i for (i,k) in enumerate(sorted(entities))}
    rel_id = {k:i for (i,k) in enumerate(sorted(relations))}
    
    if edge_filter is True:
        assert max_edges != None, 'need to specify maximum number of edges when filtering'
        connect_count = {k:0 for k in list(ent_id.keys())}
        with open(os.path.join(path, 'train.txt'), "r") as lines:
            for line in lines:
                lhs, rel, rhs = line.strip().split("\t")
                connect_count[lhs] = connect_count[lhs] + 1
                connect_count[rhs] = connect_count[rhs] + 1

#         id_to_keep = (np.where(connect_count <= max_edges)[0]).tolist() #indices of all entities with less than maximum edges
#         keep_ent = [list(ent_id.keys())[i] for i in id_to_keep] #list of entities (mids) with less than max edges 
#         del_embeddings = [k for k in list(ent_id.keys()) if k not in keep_ent] #list of deleted entities
        keep_ent = [k for k,v in connect_count.items() if v <= max_edges]
        ent_id = {k:i for i,k in enumerate(keep_ent)}
        edge_set = {}
        for split in ["train.txt", "valid.txt", "test.txt"]:
            with open(os.path.join(path, split), "r") as lines:
                edges = set()
                for line in lines:
                    lhs, rel, rhs = line.strip().split("\t")
                    if (lhs in keep_ent) and (rhs in keep_ent):
                        edges.add((lhs,rel,rhs))
            edge_set[split] = edges

               
    return ent_id, rel_id, edge_set


In [4]:
# path = '/home/shazoop/KG-Embeddings/datasets/FB15K-237'
# device = torch.device(1)

# ent_id, rel_id, edge_set = get_sets(path, edge_filter = True, max_edges = 50)

In [5]:
# def get_filtered_embeddings(path, split, D_ent, D_rel, max_edges, rel_tensor = False):
#     """
#     For each entity, generate the embedding of its neighborhood subgraph (all edges involving the entity).
    
#     input: path object, data split, entity embedding dim, relation embedding dim
#     """
#     ent_id, rel_id, ent_code, rel_code, edge_set = get_codebook(path, D_ent, D_rel)
#     N_ent = len(ent_id)

#     #Remove entities with more edges than max_edges
#     connect_count = np.zeros(N_ent)
#     if rel_tensor == True:
#         embeddings = np.zeros((N_ent,D_ent,D_ent,D_rel+1))
#     else:
#         embeddings = np.zeros((N_ent,D_ent,D_ent))
    
#     with open(os.path.join(path, split), "r") as lines:
#         for line in lines:
#             lhs, rel, rhs = line.strip().split("\t")
#             head, head_ix = ent_code[lhs], ent_id[lhs]
#             tail, tail_ix = ent_code[rhs], ent_id[rhs]
#             relation = rel_code[rel]
#             if rel_tensor == True:
#                 tensor = np.einsum('i,j,k -> ijk',head,tail,relation)
#             else:
#                 tensor = np.einsum('i,j -> ij', head, tail)
#             embeddings[head_ix] += tensor
#             embeddings[tail_ix] += tensor
#             connect_count[head_ix] += 1
#             connect_count[tail_ix] += 1
            
#     id_to_keep = (np.where(connect_count <= max_edges)[0]).tolist() #indices of all entities with less than maximum edges
#     embeddings = embeddings[id_to_keep]
#     keep_ent = [list(ent_id.keys())[i] for i in id_to_keep] #list of entities (mids) with less than max edges 
#     del_embeddings = [k for k in list(ent_id.keys()) if k not in keep_ent] #list of deleted entities
#     mod_ent_code = {k:v for k,v in ent_code.items() if k in keep_ent} 
#     return embeddings, ent_id, rel_id, mod_ent_code, rel_code, edge_set, del_embeddings, ent_code


In [6]:
def get_codebook(path, D_ent, D_rel, edge_filter = False, max_edges = None):
    """
    Generates a codebook for the set of entities and relations by randomly sampling from the unit hyperspheres of dimension
    D_ent and D_rel respectively. 
    
    input: path object, entity embedding dimension, relation embedding dimension
    
    output: dictionary of embeddings for entity and relation set (machine ids are keys for entity), plus output from get_id
    """
    ent_id, rel_id, edge_set = get_sets(path, edge_filter, max_edges)
    
    def normal_vec(dim):
        vec = np.random.multivariate_normal(np.zeros(dim),np.eye(dim))
        return vec/np.linalg.norm(vec)
    
    def nv_append1(dim):
        vec = np.random.multivariate_normal(np.zeros(dim),np.eye(dim))
        return np.append(1.,vec/np.linalg.norm(vec))

    
    ent_code = {k:normal_vec(D_ent) for k in list(ent_id.keys())}
    rel_code = {k:nv_append1(D_rel) for k in list(rel_id.keys())}
    
    return ent_id, rel_id, ent_code, rel_code, edge_set

In [7]:
# ent_id, rel_id, ent_code, rel_code, edge_set = get_codebook(path, 20, 20, edge_filter = True, max_edges = 50)

In [8]:
def get_embeddings(split, ent_id, rel_id, ent_code, rel_code, edge_set, rel_tensor = False):
    """
    For each entity, generate the embedding of its neighborhood subgraph (all edges involving the entity).
    
    input: path object, data split, entity embedding dim, relation embedding dim
    """
    N_ent = len(ent_id)
    D_ent = (list(ent_code.values())[0]).shape[0]
    D_rel = (list(rel_code.values())[0]).shape[0]
    
    if rel_tensor == True:
        embeddings = np.zeros((N_ent,D_ent,D_ent,D_rel+1))
    else:
        embeddings = np.zeros((N_ent,D_ent,D_ent))
    
    for e in list(edge_set[split]):
        head, rel, tail = e[0], e[1], e[2]
        head_ebd, rel_ebd, tail_ebd = ent_code[head], rel_code[rel], ent_code[tail]
        if rel_tensor == True:
            tensor = np.einsum('i,j,k -> ijk',head_ebd,tail_ebd,rel_ebd)
        else:
            tensor = np.einsum('i,j -> ij', head_ebd,tail_ebd)
        embeddings[ent_id[head]] += tensor
        embeddings[ent_id[tail]] += tensor
                
    return embeddings


In [9]:
# embeddings = get_embeddings('train.txt', ent_id, rel_id, ent_code, rel_code, edge_set, rel_tensor = False)

In [10]:
def easy_embeddings(path, split, D_ent, D_rel, rel_tensor = False, edge_filter = False, max_edges = None):
    """
    For each entity, generate the embedding of its neighborhood subgraph (all edges involving the entity).
    
    input: path object, data split, entity embedding dim, relation embedding dim
    """
    ent_id, rel_id, ent_code, rel_code, edge_set = get_codebook(path, D_ent, D_rel, edge_filter, max_edges)
    
    embeddings =   get_embeddings(split, ent_id, rel_id, ent_code, rel_code, edge_set, rel_tensor)
    
    return embeddings, ent_id, rel_id, ent_code, rel_code, edge_set


In [11]:
# def get_embeddings(path, split, D_ent, D_rel, rel_tensor = False, edge_filter = False, max_edges = None):
#     """
#     For each entity, generate the embedding of its neighborhood subgraph (all edges involving the entity).
    
#     input: path object, data split, entity embedding dim, relation embedding dim
#     """
#     ent_id, rel_id, ent_code, rel_code, edge_set = get_codebook(path, D_ent, D_rel, edge_filter, max_edges)
#     N_ent = len(ent_id)
    
#     if rel_tensor == True:
#         embeddings = np.zeros((N_ent,D_ent,D_ent,D_rel+1))
#     else:
#         embeddings = np.zeros((N_ent,D_ent,D_ent))
    
    
#     for e in list(edge_set[split]):
#         head, rel, tail = e[0], e[1], e[2]
#         head_ebd, rel_ebd, tail_ebd = ent_code[head], rel_code[rel], ent_code[tail]
#         if rel_tensor == True:
#             tensor = np.einsum('i,j,k -> ijk',head_ebd,tail_ebd,rel_ebd)
#         else:
#             tensor = np.einsum('i,j -> ij', head_ebd,tail_ebd)
#         embeddings[ent_id[head]] += tensor
#         embeddings[ent_id[rel]] += tensor
                
#     return embeddings, ent_id, rel_id, ent_code, rel_code, edge_set


In [12]:
# def get_embeddings(path, split, D_ent, D_rel, rel_tensor = False):
#     """
#     For each entity, generate the embedding of its neighborhood subgraph (all edges involving the entity).
    
#     input: path object, data split, entity embedding dim, relation embedding dim
#     """
#     ent_id, rel_id, ent_code, rel_code, edge_set = get_codebook(path, D_ent, D_rel)
#     N_ent = len(ent_id)
    
#     if rel_tensor == True:
#         embeddings = np.zeros((N_ent,D_ent,D_ent,D_rel+1))
#     else:
#         embeddings = np.zeros((N_ent,D_ent,D_ent))
    
#     with open(os.path.join(path, split), "r") as lines:
#         for line in lines:
#             lhs, rel, rhs = line.strip().split("\t")
#             head = ent_code[lhs]
#             tail = ent_code[rhs]
#             relation = rel_code[rel]
#             if rel_tensor == True:
#                 tensor = np.einsum('i,j,k -> ijk',head,tail,relation)
#             else:
#                 tensor = np.einsum('i,j -> ij', head, tail)
#             embeddings[ent_id[lhs]] += tensor
#             embeddings[ent_id[rhs]] += tensor
                
#     return embeddings, ent_id, rel_id, ent_code, rel_code, edge_set


In [13]:
def avg_ePn(edge_set):
    '''
    Estimates average edges per node by just dividing number of edges by number of nodes
    Input is the 'edge_set' output from any of the "get_..." functions
    '''
    edge_total = 0
    for key in list(edge_set.keys()):
        edge_total = edge_total + len(edge_set[key])
    return(math.ceil(edge_total/len(ent_id)))

In [14]:
def embed_to_device(nbd_embeddings, ent_code, rel_code, device):
    ent_embeddings = torch.from_numpy(np.stack(list(ent_code.values()),0))
    rel_embeddings = torch.from_numpy(np.stack(list(rel_code.values()),0))
    ent_embeddings = ent_embeddings.to(device)
    rel_embeddings = rel_embeddings.to(device)
    nbd_embeddings = torch.from_numpy(nbd_embeddings).to(device) 
    return nbd_embeddings, ent_embeddings, rel_embeddings

In [15]:
def tch_projmatrix(ent, ent_embeddings):
    '''
    Given the code for an entity h, will return a batch of projection matrices for each entity e.
    If h,e are vector embeddings, then will compute eh^T for each entity e. Returns the batch of these matrices.
    
    Input: ent (vector embedding), dictionary of entity vector embeddings
    
    Output: batch of projection matrices, one for each entity
    '''
    return torch.einsum('ni,j ->nij',ent_embeddings,ent)

In [16]:
# def tch_score(edge, ent_embed, ttl_ent_embed , rel_embeddings, nbd_embeddings, ent_id, rel_id, k, rel_tensor= False):
#     '''
#     Score the proposed relation (h,r,t) by using similarity matching. Assume tail is the variable.
#     we just use connectivity here and ignore relations when computing similarity
#     edges is a tuple (mid1, relation, mid2)
#     '''
#     #Get the ids/vector embeddings for head, tail, relation. Get nbd embedding for head
#     head, relation ,tail = edge[0], edge[1], edge[2]
#     head_ix, tail_ix, relation_ix = ent_id[head], ent_id[tail], rel_id[relation]
#     head_embed, tail_embed, rel_embed = ttl_ent_embed[head_ix], ttl_ent_embed[tail_ix], rel_embeddings[relation_ix]
    
#     if rel_tensor == True:
#         head_nbd = nbd_embeddings[head_ix,:,:,0] #just use edge connectivity
#         nbd_embed_norel = nbd_embeddings[:,:,:,0]
#     else:
#         head_nbd = nbd_embeddings[head_ix]
#         nbd_embed_norel = nbd_embeddings
    
#     #Generate the projection matrices for the given head
#     proj = tch_projmatrix(head_embed, ent_embed)
#     Frob_norm = torch.norm(head_nbd)**2
#     #Generate the 
    
#     #Compute the graph homomorphism coeff
#     x1 = torch.einsum('nij,jk -> nik',proj,head_nbd) #left multiply by projection matrix
#     x2 = torch.einsum('ij,nkj -> nik',head_nbd,proj) #right multiply by transpose
#     x = x1 + x2
#     res = torch.bmm(x.permute(0,2,1),nbd_embed_norel)
#     coeff = (1/Frob_norm)*torch.einsum('nii -> n',res) #number of matching edges
    
# #     #Get the top k
# #     (val,ix) = torch.topk(coeff,k)
# #     edge_coeff = torch.zeros(k-1).to(device)
# #     for i in range(1,k): #ignore top match, since that's likely to be nbd_embedding of head itself
# #         curr_ix = ix[i].cpu().item()
# #         curr_head_embed = ent_embeddings[curr_ix]
# #         edge_coeff[i-1] = torch.einsum('i,ij,j->',curr_head_embed,nbd_embeddings[curr_ix],tail_embed).cpu().item()
    
# #     #Compute score by weighting each score (-1,1) by softmax of the similarities
# #     score = torch.dot(torch.nn.functional.softmax(val[1:],dim=0).float(),edge_coeff)

#     #Get the top k
#     (val,ix) = torch.topk(coeff,k)
#     edge_coeff = torch.zeros(k).to(device)
#     for i in range(k): #ignore top match, since that's likely to be nbd_embedding of head itself
#         curr_ix = ix[i].cpu().item()
#         curr_head_embed = ent_embeddings[curr_ix]
#         edge_coeff[i] = torch.einsum('i,ij,j->',curr_head_embed,nbd_embeddings[curr_ix],tail_embed).cpu().item()
#         edge_coeff = 2*edge_coeff.clamp(min=0.,max=1.) - 1
#     #Compute score by weighting each score (-1,1) by softmax of the similarities
#     score = torch.dot(val.float(),edge_coeff)

    
#     return score, (val,ix), edge_coeff
    
    

In [17]:
def tch_score(edge,ent_embeddings, rel_embeddings, nbd_embeddings, ent_id, rel_id, k, rel_tensor= False):
    '''
    Score the proposed relation (h,r,t) by using similarity matching. Assume tail is the variable.
    we just use connectivity here and ignore relations when computing similarity
    edges is a tuple (mid1, relation, mid2)
    '''
    #Get the ids/vector embeddings for head, tail, relation. Get nbd embedding for head
    head, relation ,tail = edge[0], edge[1], edge[2]
    head_ix, tail_ix, relation_ix = ent_id[head], ent_id[tail], rel_id[relation]
    head_embed, tail_embed, rel_embed = ent_embeddings[head_ix], ent_embeddings[tail_ix], rel_embeddings[relation_ix]
    
    if rel_tensor == True:
        head_nbd = nbd_embeddings[head_ix,:,:,0] #just use edge connectivity
        nbd_embed_norel = nbd_embeddings[:,:,:,0]
    else:
        head_nbd = nbd_embeddings[head_ix]
        nbd_embed_norel = nbd_embeddings
    
    #Generate the projection matrices for the given head
    proj = tch_projmatrix(head_embed, ent_embeddings)
    Frob_norm_sq = torch.norm(head_nbd)**2
    #Generate the 
    
    #Compute the graph homomorphism coeff
    x1 = torch.einsum('nij,jk -> nik',proj,head_nbd) #left multiply by projection matrix
    x2 = torch.einsum('ij,nkj -> nik',head_nbd,proj) #right multiply by transpose projection
    x = x1 + x2
    res = torch.bmm(x.permute(0,2,1),nbd_embed_norel)
    coeff = (1/(Frob_norm_sq + 1e-3))*torch.einsum('nii -> n',res) #number of matching edges
    
    #Get the top k
    (val,ix) = torch.topk(coeff,k)
    edge_coeff = torch.zeros(k-1).to(device)
    for i in range(1,k): #ignore top match, since that's likely to be nbd_embedding of head itself
        curr_ix = ix[i].cpu().item()
        edge_coeff[i-1] = torch.einsum('i,ij,j->',ent_embeddings[curr_ix],nbd_embeddings[curr_ix],tail_embed).cpu().item()
#         edge_coeff = 2*edge_coeff.clamp(min=0.,max=1.) - 1
    
    #Compute score by weighting each score (-1,1) by softmax of the similarities
    score = torch.dot(val[1:].float(),edge_coeff)
    
    return score, (val,ix), Frob_norm_sq

In [18]:
def test(num_batch, ent_embeddings, rel_embeddings, nbd_embeddings, ent_id, rel_id, k, edge_set, rel_tensor= False):
    edges = list(edge_set['test.txt'])
    batch = random.sample(edges,num_batch)
    ttl_score = 0
    num_edges = 0
    
    for e in batch:
        score, _ , Frob_norm_sq= tch_score(e,ent_embeddings, rel_embeddings, nbd_embeddings, ent_id, rel_id, k, rel_tensor)
        if Frob_norm_sq <= 1e-3:
            continue
        ttl_score = ttl_score + score.cpu().item()
        num_edges = num_edges + 1
    
    return ttl_score/num_edges
    

In [19]:
path = '/home/shazoop/KG-Embeddings/datasets/FB15K-237'
device = torch.device(1)

In [46]:
embeddings, ent_id, rel_id, ent_code, rel_code, edge_set = easy_embeddings(path, 'train.txt', D_ent = 80, D_rel = 80, rel_tensor = False, edge_filter = True, max_edges = 30)

In [47]:
nbd_embeddings, ent_embeddings, rel_embeddings = embed_to_device(embeddings, ent_code, rel_code, device)

In [48]:
len(ent_id)

7298

In [21]:
edge = list(edge_set['test.txt'])[600]

NameError: name 'edge_set' is not defined

In [20]:
tch_score(edge,ent_embeddings, rel_embeddings, nbd_embeddings, ent_id, rel_id, 10, rel_tensor= False)

NameError: name 'edge' is not defined

In [107]:
test(500, ent_embeddings, rel_embeddings, nbd_embeddings, ent_id, rel_id, 10, edge_set, rel_tensor= False)

0.11828963966609685

# Junk

In [ ]:
ttl_ent_embed = ttl_ent_embeddings
head, relation ,tail = edge[0], edge[1], edge[2]
head_ix, tail_ix, relation_ix = ent_id[head], ent_id[tail], rel_id[relation]
head_embed, tail_embed, rel_embed = ttl_ent_embed[head_ix], ttl_ent_embed[tail_ix], rel_embeddings[relation_ix]

if rel_tensor == True:
    head_nbd = nbd_embeddings[head_ix,:,:,0] #just use edge connectivity
    nbd_embed_norel = nbd_embeddings[:,:,:,0]
else:
    head_nbd = nbd_embeddings[head_ix]
    nbd_embed_norel = nbd_embeddings

#Generate the projection matrices for the given head
proj = tch_projmatrix(head_embed, ent_embed)
Frob_norm = torch.norm(head_nbd)**2


In [ ]:
nbd_graph = nbd_embeddings[ent_id[edge[0]]]

In [ ]:
torch.norm(nbd_graph)

In [ ]:
head_embed = ent_embeddings[0]
tail_embed = ent_embeddings[3000]

In [ ]:
proj = tch_projmatrix(head_embed, ent_embeddings)
head_nbd = nbd_embeddings[0]
Frob_norm = torch.norm(head_nbd)**2

In [ ]:
proj1 = torch.einsum('i,j -> ij', head_embed, head_embed)

In [ ]:
torch.mm(torch.mm(proj1,head_nbd),proj1.t())

In [ ]:
torch.norm(head_nbd-x[0])

In [ ]:
head_nbd

In [ ]:
x[0]

In [ ]:
x = torch.einsum('nij,jk -> nik',proj,head_nbd) #left multiply by projection matrix
x = torch.einsum('nij,nkj -> nik',x,proj) #right multiply by transpose
res = torch.bmm(x.permute(0,2,1),nbd_embeddings)
coeff = (1/Frob_norm)*torch.einsum('nii -> n',res) #number of matching edges

In [ ]:
k = 20
(val,ix) = torch.topk(coeff,k)
edge_coeff = torch.zeros(k-1).to(device)
for i in range(1,k): #ignore top match, since that's likely to be nbd_embedding of head itself
    curr_ix = ix[i].cpu().item()
    edge_coeff[i-1] = torch.einsum('i,ij,j->',head_embed,nbd_embeddings[curr_ix],tail_embed).cpu().item()

In [ ]:
(val,ix)

In [ ]:
edge_list = list(sorted(edge_set['train.txt']))

In [ ]:
problem_id = list(ent_id.keys())[5625]

In [ ]:
torch.norm(nbd_embeddings[5625])**2

# Testing Graph Reps

In [ ]:
head_embed = ent_embeddings[0]
tail_embed = ent_embeddings[10:30]
new_embed = ent_embeddings[31:50]

In [ ]:
nbd_test = torch.einsum('i,nj -> ij',head_embed,tail_embed)

In [ ]:
torch.norm(nbd_test)**2

In [ ]:
torch.einsum('i,ij,nj ->n',head_embed,nbd_test,tail_embed)

In [ ]:
torch.einsum('i,ij,nj ->n',head_embed,nbd_test,new_embed)

In [ ]:
torch.norm(torch.mm(nbd_test.t(),nbd_test))/torch.norm(nbd_test)**2

In [ ]:
proj_mat = torch.einsum('i,j -> ij',head_embed, head_embed)

In [ ]:
x_test1 = torch.einsum('ij,jk -> ik',proj_mat,nbd_test) #left multiply by projection matrix
x_test2 = torch.einsum('ij,kj -> ik', nbd_test,proj_mat) #right multiply by transpose
x_test = x_test1 + x_test2

In [ ]:
res_test = torch.mm(x_test.t(),nbd_test)
coeff = (1/(torch.norm(nbd_test)**2))*torch.einsum('ii -> ',res_test) #number of matching edges
coeff